# Airlines Review


---------------------

## Phase 3 - LLM Mistral


### Environment setup

Setting Up Hugging Face to Use the E: Drive instead of the default C: drive.
This saves local disk space and helps manage large files better.

In [1]:
# !pip install transformers datasets peft accelerate bitsandbytes 
import os

# Store all Hugging Face files on the E: drive
os.environ["HF_HOME"] = "E:/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "E:/huggingface/transformers"
os.environ["HF_DATASETS_CACHE"] = "E:/huggingface/datasets"


In [2]:
import torch
print(torch.cuda.get_device_name(0))
print(f"Total VRAM: {round(torch.cuda.get_device_properties(0).total_memory / 1e9, 2)} GB")

NVIDIA GeForce GTX 1660 Ti
Total VRAM: 6.44 GB


### Load the model

The TinyMistral-248M model is loaded and prepared for efficient fine-tuning:

- First, 4-bit quantization is set up using BitsAndBytesConfig to save GPU memory.

- The tokenizer and base model are loaded from HuggingFace, configured for a 3-class classification task.

- The model is then adapted for 4-bit training with prepare_model_for_kbit_training.

- Finally, LoRA (Low-Rank Adaptation) is applied by injecting lightweight trainable layers into the attention mechanisms (q_proj and v_proj), making the fine-tuning process much faster and lighter.

- The padding token is also corrected after these adjustments to ensure input sequences are properly handled.

In [3]:
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForSequenceClassification
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig, TaskType

# Setup 4-bit Quantization : configure the model to load in 4-bit precision to save memory (important with small VRAM GPUs like 6 GB).
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

#  Load Tokenizer
model_id = "Locutusque/TinyMistral-248M"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  #  Add padding

# Base model: Load Pre-trained Mistral Model TinyMistral
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    num_labels=3, # ready for classification tasks with 3 output classes
    trust_remote_code=True
).to("cuda")

# ✅ Prepare for LoRA after quant: adapt the model for training in 4-bit precision, making it faster and lighter to fine-tune.
model = prepare_model_for_kbit_training(model)

# Apply LoRA Fine-Tuning : configure LoRA (Low-Rank Adaptation) to inject small, efficient trainable adapters into the model’s attention layers.
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)
model = get_peft_model(model, lora_config)

# Now apply pad_token_id after PEFT
model.config.pad_token_id = tokenizer.pad_token_id

print("Pad token:", tokenizer.pad_token)
print("Pad token ID:", tokenizer.pad_token_id)


c:\Users\Alicia\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\utils\hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at Locutusque/TinyMistral-248M and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Pad token: <|endoftext|>
Pad token ID: 32001


### Load and Preprocess the Dataset

#### Loading the dataset:

In [4]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

# Load the dataset
df = pd.read_csv("./AirlinesReviews/data_phase1.csv", encoding='utf-8', on_bad_lines='skip')
df.head()

,Name,Review Date,Airline,Verified,Type of Traveller,Month Flown,Route,Class,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended,Review,Sentiment,Review_Length
0,Alison Soetantyo,2024-03-01,Singapore Airlines,True,Solo Leisure,December 2023,Jakarta to Singapore,Business Class,4,4,4,4,4,9,yes,Flight was amazing. Flight was amazing. The ...,Positive,89
1,Robert Watson,2024-02-21,Singapore Airlines,True,Solo Leisure,February 2024,Phuket to Singapore,Economy Class,5,3,4,4,1,3,no,seats on this aircraft are dreadful . Bookin...,Negative,49
2,S Han,2024-02-20,Singapore Airlines,True,Family Leisure,February 2024,Siem Reap to Singapore,Economy Class,1,5,2,1,5,10,yes,Food was plentiful and tasty. Excellent perf...,Positive,34
3,D Laynes,2024-02-19,Singapore Airlines,True,Solo Leisure,February 2024,Singapore to London Heathrow,Economy Class,5,5,5,5,5,10,yes,“how much food was available. Pretty comforta...,Positive,171
4,A Othman,2024-02-19,Singapore Airlines,True,Family Leisure,February 2024,Singapore to Phnom Penh,Economy Class,5,5,5,5,5,10,yes,“service was consistently good”. The service ...,Positive,57


#### Preprocessing

The dataset is cleaned and prepared by mapping sentiment labels (Positive, Negative, Neutral) to numerical values required for model training. 

Only the review text and corresponding label were retained, with the text column renamed to text for compatibility with the tokenizer. 

The "Review" column is renamed to "text" to match HuggingFace standards.

The dataset is then split into training, validation, and test sets following an 80%-10%-10% split, ensuring random and reproducible partitions for fine-tuning the Mistral model.

This simplified structure ensures the data is ready for tokenization and fine-tuning with the Mistral model.

In [ ]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import pandas as pd

#  Map sentiment labels to integers
label_map = {'Negative': 0, 'Positive': 1, 'Neutral': 2}
df = df[df['Sentiment'].isin(label_map.keys())]
df['label'] = df['Sentiment'].map(label_map)
df = df[['Review', 'label']].rename(columns={'Review': 'text'})

# === STEP 1: Split original imbalanced data FIRST ===
train_df, test_df = train_test_split(
    df,
    test_size=0.1,
    stratify=df['label'],  # maintain class ratio in test
    random_state=42
)

# === STEP 2: Upsample Neutral class ONLY in training set ===
df_neg = train_df[train_df['label'] == 0]
df_pos = train_df[train_df['label'] == 1]
df_neu = train_df[train_df['label'] == 2]

# Match to largest class
target_size = max(len(df_neg), len(df_pos))
df_neu_upsampled = resample(
    df_neu,
    replace=True,
    n_samples=target_size,
    random_state=42
)

# Combine balanced training set
balanced_train_df = pd.concat([df_neg, df_pos, df_neu_upsampled]).sample(frac=1, random_state=42).reset_index(drop=True)

# === STEP 3: Convert to Hugging Face datasets ===
train_dataset = Dataset.from_pandas(balanced_train_df)
test_dataset = Dataset.from_pandas(test_df)

# === STEP 4: Split train into train/validation (10% val) ===
train_val_split = train_dataset.train_test_split(test_size=0.1111, seed=42)

# === STEP 5: Wrap in DatasetDict ===
dataset = DatasetDict({
    "train": train_val_split["train"],
    "validation": train_val_split["test"],
    "test": test_dataset  # ✅ original imbalanced test
})


### Tokenize the Dataset

The dataset is tokenized using the same tokenizer as the Mistral model:

A custom tokenize function is applied to the "text" column, ensuring all sequences are padded or truncated to a maximum length of 512 tokens.
The dataset is processed in batches for faster tokenization, preparing the text inputs for model training.

In [6]:
from transformers import AutoTokenizer
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)



Map:   0%|          | 0/7866 [00:00<?, ? examples/s]

Map:   0%|          | 0/984 [00:00<?, ? examples/s]

Map:   0%|          | 0/810 [00:00<?, ? examples/s]

### Hyperparameter Search with Optuna

Hyperparameter optimization is performed using Optuna:

- The model is trained multiple times on the training set, and evaluated on the validation set, with the goal of maximizing the weighted F1-score.

- Optuna explores different learning rates, batch sizes, and numbers of epochs to automatically find the best combination.

- Training is monitored with a custom callback to track progress, and early stopping is used to avoid overfitting.

- To address class imbalance in the dataset, class weights are computed automatically based on the frequency of each emotion label (Positive, Negative, Neutral).
These weights are applied during training using a custom WeightedTrainer that modifies the loss function.
This ensures that the model gives more importance to underrepresented classes and does not bias predictions toward majority classes.

In [8]:
import time
import optuna
import numpy as np
import torch
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight

from transformers import (
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainerCallback,
    EarlyStoppingCallback,
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig
from torch import nn
from transformers import Trainer


#  Class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(df["label"]),
    y=df["label"]
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to("cuda")

#  TRACK TIME PER TRIAL
class TrialProgressCallback(TrainerCallback):
    def __init__(self):
        self.start_time = None

    def on_train_begin(self, args, state, control, **kwargs):
        self.start_time = time.time()
        print(f"\n🚀 Starting trial at {time.strftime('%H:%M:%S')}")

    def on_log(self, args, state, control, logs=None, **kwargs):
        elapsed = time.time() - self.start_time
        print(f"⏱️ Step {state.global_step} - Elapsed: {round(elapsed/60, 2)} min - Logs: {logs}")

#  METRICS
def compute_metrics(pred):
    preds = pred.predictions.argmax(-1)
    return {"f1": f1_score(pred.label_ids, preds, average="weighted")}

#  MODEL INIT (with LoRA)
def model_init():
    base_model = AutoModelForSequenceClassification.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        num_labels=3,
        trust_remote_code=True
    )
    base_model.config.pad_token_id = tokenizer.pad_token_id
    base_model = prepare_model_for_kbit_training(base_model)

    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.SEQ_CLS
    )
    lora_model = get_peft_model(base_model, lora_config)
    return lora_model

#  OPTUNA SEARCH SPACE
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 2e-5, 5e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2, 4]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 3)
    }

#  Custom Trainer with class weights
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss



#  TRAINING ARGS
training_args = TrainingArguments(
    output_dir="./optuna_output",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    report_to="none",
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=100,
    fp16=False
)

#  TRAINER with everything
trainer = WeightedTrainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[
        TrialProgressCallback(),
        EarlyStoppingCallback(early_stopping_patience=2)
    ]
)

#  RUN OPTUNA
start = time.time()

best_trial = trainer.hyperparameter_search(
    direction="maximize",
    hp_space=optuna_hp_space,
    n_trials=3
)

end = time.time()

print("\n✅ Done!")
print(f"⏱️ Optuna tuning done in {round((end - start)/60, 2)} minutes")
print("🏆 Best trial params:", best_trial.hyperparameters)


C:\Users\Alicia\AppData\Local\Temp\ipykernel_20304\1254717194.py:105: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(
Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at Locutusque/TinyMistral-248M and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
[I 2025-05-27 17:37:55,856] A new study created in memory with name: no-name-0a1b9c76-415a-4624-be96-8b3d56d59d15
Some weights of MistralForSequenceClassification were not initialized from the model checkp


🚀 Starting trial at 17:37:56


c:\Users\Alicia\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,F1
1,1.201600,0.919895,0.760713
2,0.849500,0.905518,0.788788
3,0.870300,0.886684,0.794574


⏱️ Step 100 - Elapsed: 0.9 min - Logs: {'loss': 1.1162, 'grad_norm': 17.354602813720703, 'learning_rate': 7.317371887170678e-05, 'epoch': 0.025425883549453344}
⏱️ Step 200 - Elapsed: 1.76 min - Logs: {'loss': 1.086, 'grad_norm': 8.703713417053223, 'learning_rate': 7.254830247109389e-05, 'epoch': 0.05085176709890669}
⏱️ Step 300 - Elapsed: 2.61 min - Logs: {'loss': 0.9988, 'grad_norm': 20.841867446899414, 'learning_rate': 7.1922886070481e-05, 'epoch': 0.07627765064836003}
⏱️ Step 400 - Elapsed: 3.47 min - Logs: {'loss': 0.9255, 'grad_norm': 24.065855026245117, 'learning_rate': 7.129746966986813e-05, 'epoch': 0.10170353419781338}
⏱️ Step 500 - Elapsed: 4.33 min - Logs: {'loss': 0.8775, 'grad_norm': 22.398548126220703, 'learning_rate': 7.067205326925526e-05, 'epoch': 0.12712941774726672}
⏱️ Step 600 - Elapsed: 5.19 min - Logs: {'loss': 0.8477, 'grad_norm': 5.368893623352051, 'learning_rate': 7.004663686864238e-05, 'epoch': 0.15255530129672007}
⏱️ Step 700 - Elapsed: 6.05 min - Logs: {'los

...

⏱️ Step 4000 - Elapsed: 35.7 min - Logs: {'loss': 0.6478, 'grad_norm': 0.14166556298732758, 'learning_rate': 0.000116667627116227, 'epoch': 1.0170353419781337}
⏱️ Step 4100 - Elapsed: 36.56 min - Logs: {'loss': 0.6961, 'grad_norm': 1.086085557937622, 'learning_rate': 0.00011517188830704459, 'epoch': 1.0424612255275871}
⏱️ Step 4200 - Elapsed: 37.43 min - Logs: {'loss': 0.8022, 'grad_norm': 9.959230422973633, 'learning_rate': 0.0001136761494978622, 'epoch': 1.0678871090770405}
⏱️ Step 4300 - Elapsed: 38.3 min - Logs: {'loss': 0.749, 'grad_norm': 0.3586465120315552, 'learning_rate': 0.0001121804106886798, 'epoch': 1.0933129926264937}
⏱️ Step 4400 - Elapsed: 39.16 min - Logs: {'loss': 0.5676, 'grad_norm': 0.9739956855773926, 'learning_rate': 0.0001106846718794974, 'epoch': 1.1187388761759471}
⏱️ Step 4500 - Elapsed: 40.03 min - Logs: {'loss': 0.7901, 'grad_norm': 40.108070373535156, 'learning_rate': 0.00010918893307031501, 'epoch': 1.1441647597254005}
⏱️ Step 4600 - Elapsed: 40.9 min - Lo

c:\Users\Alicia\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


⏱️ Step 7900 - Elapsed: 70.53 min - Logs: {'loss': 0.602, 'grad_norm': 0.09784363210201263, 'learning_rate': 5.83338135581135e-05, 'epoch': 2.008644800406814}
⏱️ Step 8000 - Elapsed: 71.4 min - Logs: {'loss': 0.641, 'grad_norm': 0.08299805968999863, 'learning_rate': 5.68380747489311e-05, 'epoch': 2.0340706839562674}
⏱️ Step 8100 - Elapsed: 72.26 min - Logs: {'loss': 0.5462, 'grad_norm': 0.05765227600932121, 'learning_rate': 5.53423359397487e-05, 'epoch': 2.059496567505721}
⏱️ Step 8200 - Elapsed: 73.12 min - Logs: {'loss': 0.6828, 'grad_norm': 0.02796316333115101, 'learning_rate': 5.384659713056631e-05, 'epoch': 2.0849224510551743}
⏱️ Step 8300 - Elapsed: 73.98 min - Logs: {'loss': 0.7305, 'grad_norm': 0.033547017723321915, 'learning_rate': 5.23508583213839e-05, 'epoch': 2.1103483346046277}
⏱️ Step 8400 - Elapsed: 74.84 min - Logs: {'loss': 0.5378, 'grad_norm': 11.05135726928711, 'learning_rate': 5.085511951220151e-05, 'epoch': 2.135774218154081}
⏱️ Step 8500 - Elapsed: 75.7 min - Logs

[I 2025-05-27 23:14:13,496] Trial 2 finished with value: 0.828066301754496 and parameters: {'learning_rate': 0.00017648222209543107, 'per_device_train_batch_size': 2, 'num_train_epochs': 3}. Best is trial 2 with value: 0.828066301754496.



✅ Done!
⏱️ Optuna tuning done in 336.29 minutes
🏆 Best trial params: {'learning_rate': 0.00017648222209543107, 'per_device_train_batch_size': 2, 'num_train_epochs': 3}


### Saving the best parameters

In [9]:
print("🏆 Best trial F1 score:", best_trial.objective)
print("📋 Best trial hyperparameters:", best_trial.hyperparameters)

🏆 Best trial F1 score: 0.828066301754496
📋 Best trial hyperparameters: {'learning_rate': 0.00017648222209543107, 'per_device_train_batch_size': 2, 'num_train_epochs': 3}


In [10]:
import json
# Save best params
with open("best_params_MISTRAL-v4.json", "w") as f:
    json.dump(best_trial.hyperparameters, f)

print("✅ Best hyperparameters saved.")


✅ Best hyperparameters saved.


### Final model

#### Merging Train and Validation Sets

The training and validation datasets are merged into a single full training set.
This allows the final model to be trained using all available labeled data for better performance, instead of wasting examples on separate validation.

In [7]:
# Merge train and validation splits
full_train_dataset = Dataset.from_dict({
    key: tokenized_dataset["train"][key] + tokenized_dataset["validation"][key]
    for key in tokenized_dataset["train"].features
})


### Final version

In this final version, the model is fine-tuned using the best hyperparameters previously found with Optuna.

Class imbalance is addressed by assigning a higher weight to the Neutral class during training.

A custom WeightedTrainer is used to apply class weights correctly through a modified loss function (CrossEntropyLoss).

The model is initialized with LoRA (Low-Rank Adaptation) on top of TinyMistral-248M, using 4-bit quantization to optimize memory usage.

Training is done on the full training dataset, with no evaluation during training (evaluation_strategy="no"), to prevent data leakage.

After training, predictions are made on the separate test set, and a threshold optimization is applied to better distinguish Neutral class predictions.

Finally, classification metrics, a confusion matrix, and the weighted F1-score are printed to summarize the model's performance.

In [8]:
#  Imports
import torch
import json
import numpy as np
import time
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from torch import nn

#  Load Best Parameters
with open("best_params_MISTRAL.json", "r") as f:
    best_params = json.load(f)



#  Class weights
class_weights = torch.tensor([1.0, 1.0, 2.0], dtype=torch.float).to("cuda")  # Neutral weighted higher


#  Custom Trainer to inject class weights
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

#  Model Init function
def model_init():
    base_model = AutoModelForSequenceClassification.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        num_labels=3,
        trust_remote_code=True
    )
    base_model.config.pad_token_id = tokenizer.pad_token_id
    base_model = prepare_model_for_kbit_training(base_model)
    
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.SEQ_CLS
    )
    model = get_peft_model(base_model, lora_config)
    return model

#  Training Args
final_training_args = TrainingArguments(
    output_dir="./final",
    eval_strategy="no",
    save_strategy="no",  
    learning_rate=best_params["learning_rate"],
    per_device_train_batch_size=best_params["per_device_train_batch_size"],
    num_train_epochs=3,
    report_to="none",
    fp16=False
)

#  Final Trainer
final_trainer = WeightedTrainer(
    model_init=model_init,
    args=final_training_args,
    train_dataset=full_train_dataset,  # <<< Full train
    tokenizer=tokenizer
)

#  Train
start = time.time()
final_trainer.train()
end = time.time()
print(f"\n✅ Final training completed in {round((end-start)/60, 2)} minutes.")

#  Save Model
final_trainer.save_model("finalBALANCE")
print(" Final model saved to 'finalBALANCE' folder.")

#  Predictions
predictions = final_trainer.predict(tokenized_dataset["test"])
probs = torch.softmax(torch.tensor(predictions.predictions), dim=1).numpy()
labels = predictions.label_ids

#  Auto Threshold Optimization
from scipy.optimize import minimize_scalar

def threshold_objective(thresh):
    preds = np.argmax(probs, axis=1)
    max_probs = np.max(probs, axis=1)
    preds[max_probs < thresh] = 2  # Force to Neutral if uncertain
    return -f1_score(labels, preds, average="weighted")

opt_result = minimize_scalar(threshold_objective, bounds=(0.3, 0.7), method="bounded")
best_thresh = opt_result.x
print(f"\n🔍 Best Neutral Threshold found: {round(best_thresh, 3)}")

#  Apply Best Threshold
preds = np.argmax(probs, axis=1)
max_probs = np.max(probs, axis=1)
preds[max_probs < best_thresh] = 2  # Again force Neutral

#  Print Metrics
print("\n📘 Classification Report (Final):")
print(classification_report(labels, preds, target_names=['Negative', 'Positive', 'Neutral']))

print("\n✅ Confusion Matrix:")
print(confusion_matrix(labels, preds))

final_f1 = f1_score(labels, preds, average="weighted")
print(f"\n🔴 Final Weighted F1 (with threshold): {round(final_f1, 4)}")


C:\Users\Alicia\AppData\Local\Temp\ipykernel_2020\3935187034.py:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  final_trainer = WeightedTrainer(
Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at Locutusque/TinyMistral-248M and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at Locutusque/TinyMistral-248M and are newly initialized: ['score.weight']
You should probably TRAIN this

Step,Training Loss
500,0.899100
1000,0.863000
1500,0.906200
2000,0.855300
2500,0.836100
3000,0.921800
3500,0.900500
4000,0.923600
4500,0.782200
5000,0.727100



✅ Final training completed in 116.38 minutes.
 Final model saved to 'finalBALANCE' folder.



🔍 Best Neutral Threshold found: 0.341

📘 Classification Report (Final):
              precision    recall  f1-score   support

    Negative       0.80      0.81      0.80       302
    Positive       0.85      0.88      0.87       341
     Neutral       0.44      0.40      0.42       167

    accuracy                           0.75       810
   macro avg       0.70      0.70      0.70       810
weighted avg       0.75      0.75      0.75       810


✅ Confusion Matrix:
[[245   3  54]
 [ 11 299  31]
 [ 52  48  67]]

🔴 Final Weighted F1 (with threshold): 0.7504


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from scipy.optimize import minimize_scalar

# === Predict on test set ===
predictions = final_trainer.predict(tokenized_dataset["test"])
logits = predictions.predictions
labels = predictions.label_ids

# === Apply softmax with temperature scaling ===
temperature = 2 
probs = F.softmax(torch.tensor(logits / temperature), dim=-1).numpy()

# === Define threshold optimization using confidence margin ===
def threshold_objective(margin):
    final_preds = []
    for p in probs:
        top2 = np.sort(p)[-2:]        # take two highest probs
        gap = top2[1] - top2[0]       # confidence gap
        if gap < margin:
            final_preds.append(2)     # force Neutral
        else:
            final_preds.append(np.argmax(p))
    return -f1_score(labels, final_preds, average="weighted")

# === Find best neutral margin ===
opt_result = minimize_scalar(threshold_objective, bounds=(0.05, 0.4), method="bounded")
neutral_margin = opt_result.x
print(f"\n🔍 Best Neutral Confidence Margin found: {round(neutral_margin, 3)}")

# === Apply margin to get final predictions ===
final_preds = []
for p in probs:
    top2 = np.sort(p)[-2:]
    gap = top2[1] - top2[0]
    if gap < neutral_margin:
        final_preds.append(2)
    else:
        final_preds.append(np.argmax(p))

# === Evaluation ===
print("\n📘 Classification Report (Confidence + Temperature):")
print(classification_report(labels, final_preds, target_names=["Negative", "Positive", "Neutral"]))

print("\n✅ Confusion Matrix:")
print(confusion_matrix(labels, final_preds))

final_f1 = f1_score(labels, final_preds, average="weighted")
print(f"\n🔴 Final Weighted F1 (with threshold): {round(final_f1, 4)}")



🔍 Best Neutral Confidence Margin found: 0.157

📘 Classification Report (Confidence + Temperature):
              precision    recall  f1-score   support

    Negative       0.81      0.79      0.80       302
    Positive       0.86      0.87      0.86       341
     Neutral       0.44      0.43      0.43       167

    accuracy                           0.75       810
   macro avg       0.70      0.70      0.70       810
weighted avg       0.75      0.75      0.75       810


✅ Confusion Matrix:
[[240   3  59]
 [ 10 297  34]
 [ 48  47  72]]

🔴 Final Weighted F1 (with threshold): 0.7512


In [15]:
from sklearn.metrics import accuracy_score

final_accuracy = accuracy_score(labels, preds)

print(f"\n✅ Final Accuracy: {final_accuracy:.4f}")
print(f"✅ Final Weighted F1-score: {final_f1:.4f}")



✅ Final Accuracy: 0.7543
✅ Final Weighted F1-score: 0.7512
